<a href="https://colab.research.google.com/github/Fmarcelneves/Rental-price-analysis-and-forecast-Curitiba-PR/blob/main/Data_collection_Curitiba_rent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import packages
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from time import sleep

In [ ]:
# first 6 pages from the website
paginas = ['']#'_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', '_Desde_241']
bairros = ['Abranches', 'Ahú', 'Alto Boqueirão', 'Alto da Glória', 'Alto da Rua XV', 'Atuba', 
           'Bacacheri', 'Bairro Alto', 'Batel', 'Bigorrilho', 'Bom Retiro', 'Boqueirão', 'Butiatuvinha',
           'Cabral', 'Cachoeira', 'Cajurú', 'Campina do Siqueira', 'Campo Comprido', 'Campo de Santana', 'Capão Raso', 'Capão da Imbuia',
           'Cascatinha','Centro', 'Centro Civíco', 'Champagnat', 'Cidade Industrial', 'Cristo Rei', 'Fanny', 'Fazendinha',
           'Guabirotuba', 'Guaíra', 'Hauer', 'Hugo Lange', 'Jardim Botânico', 'Jardim das Américas', 'Jardim Social', 'Juvevê',
           'Lindóia', 'Mercês', 'Mossunguê', 'Novo Mundo', 'Orleans', 'Parolin', 'Pilarzinho', 'Pinheirinho', 'Portão', 'Prado Velho',
           'Rebouças', 'Santa Cândida', 'Santa Felicidade', 'Santa Quitéria', 'Santo Inácio', 'Seminário', 'São Lourenço', 'São Braz', 'São Francisco', 'Sítio Cercado',
           'Tarumã', 'Tatuquara', 'Tingui', 'Uberaba', 'Umbará', 'Vila Izabel', 'Vista Alegre', 'Xaxim', 'Água Verde']

In [ ]:
def coleta_dados(url, bairro):
  conteudo = requests.get(url).content
  sopa = BeautifulSoup(conteudo)

  lista_precos = []
  lista_areas_quartos = []
  lista_quartos = []
  lista_dimensoes = []
  lista_bairros = []

  re_quartos = '(.) quarto|quartos'
  re_dimensoes = '(.*) m²'

  precos = sopa.find_all('span', class_='price-tag-fraction')
  dimensoes_e_quartos = sopa.find_all('ul', class_='ui-search-card-attributes ui-search-item__group__element')

  for elemento in precos:
    preco = re.findall('<span class="price-tag-fraction">(.*)</span>', str(elemento))
    lista_precos.append(float(preco[0].replace('.','')))


  for elemento in dimensoes_e_quartos:
    lista_areas_quartos.append(elemento.text)

  for elemento in lista_areas_quartos:
    quarto = re.findall(re_quartos, str(elemento))
    dimensao = re.findall(re_dimensoes, str(elemento))

    if quarto == []:
      quarto = np.nan
    else:
      quarto = quarto[0]
    
    if dimensao == []:
      dimensao = np.nan
    else:
      dimensao = dimensao[0]
  
    lista_quartos.append(quarto)
    lista_dimensoes.append(dimensao)
    lista_bairros.append(bairro)

  dados = {'bairro': lista_bairros,
           'precos':lista_precos,
           'quartos':lista_quartos,
           'areas': lista_dimensoes}
  
  df = pd.DataFrame(dados)
  return df

In [ ]:
X = 1.230

In [ ]:
float(X)*1000

1230.0

In [ ]:
lista_dfs = []

for bairro in bairros:
  for pagina in paginas:
    url = f'https://imoveis.mercadolivre.com.br/apartamentos/aluguel/parana/curitiba/{bairro}/{pagina}'
    print('Coletando dados de', url)
    df = coleta_dados(url, bairro)
    lista_dfs.append(df)
    sleep(2)
print('Dados coletados!')

Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/parana/curitiba/Abranches/
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/parana/curitiba/Ahú/
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/parana/curitiba/Alto Boqueirão/
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/parana/curitiba/Alto da Glória/
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/parana/curitiba/Alto da Rua XV/
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/parana/curitiba/Atuba/
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/parana/curitiba/Bacacheri/
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/parana/curitiba/Bairro Alto/
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/parana/curitiba/Batel/
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/al

In [ ]:
df_ml = pd.concat(lista_dfs)

In [ ]:
df_ml

,bairro,precos,quartos,areas
0,Abranches,960.0,1,50
1,Abranches,862.0,1,40
2,Abranches,855.0,1,28
3,Abranches,855.0,1,28
4,Abranches,855.0,1,28
...,...,...,...,...
13,Xaxim,750.0,1,30
14,Xaxim,1100.0,3,100
15,Xaxim,1000.0,2,67
16,Xaxim,1100.0,2,54


In [ ]:
df_ml.to_csv('dados_mercado_livre_ap_curitiba.csv', index=False)

In [ ]:
df_ml.describe()

,precos
count,1043.000000
mean,260.193969
std,376.382200
min,1.000000
25%,1.300000
50%,2.100000
75%,694.500000
max,995.000000


In [ ]:
da_ml = pd.read_csv('dados_mercado_livre_ap_curitiba.csv')

In [ ]:
da_ml.describe().T

,count,mean,std,min,25%,50%,75%,max
precos,1043.0,260.193969,376.382200,1.0,1.3,2.1,694.5,995.0
quartos,1038.0,2.081888,1.028072,1.0,1.0,2.0,3.0,8.0


In [ ]:
da_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1002 entries, 0 to 1001
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   bairro   1002 non-null   object 
 1   precos   1002 non-null   float64
 2   quartos  997 non-null    float64
 3   areas    999 non-null    object 
dtypes: float64(2), object(2)
memory usage: 31.4+ KB


In [ ]:
da_ml.replace(',','', regex=True,inplace=True)
da_ml['areas'] = da_ml['areas'].astype(float)

In [ ]:
da_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1002 entries, 0 to 1001
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   bairro   1002 non-null   object 
 1   precos   1002 non-null   float64
 2   quartos  997 non-null    float64
 3   areas    999 non-null    float64
dtypes: float64(3), object(1)
memory usage: 31.4+ KB


In [ ]:
da_ml.isnull().sum()

bairro     0
precos     0
quartos    5
areas      3
dtype: int64

In [ ]:
da_ml['bairro'].value_counts()

Centro                 239
Bigorrilho              70
Cidade Industrial       67
Santa Quitéria          42
Santa Cândida           42
Cristo Rei              39
Batel                   35
Novo Mundo              33
Mercês                  30
Campo Comprido          29
Vila Izabel             26
Bacacheri               24
Cabral                  23
Juvevê                  22
Pinheirinho             22
Uberaba                 20
Seminário               19
Tingui                  18
Bairro Alto             17
Xaxim                   17
Mossunguê               16
Atuba                   14
Fazendinha              11
Prado Velho             11
Hauer                   11
Guabirotuba              9
Vista Alegre             9
Pilarzinho               9
Abranches                9
Jardim Social            8
Alto da Rua XV           7
Santa Felicidade         7
Orleans                  6
Santo Inácio             6
Cachoeira                6
Bom Retiro               5
Campina do Siqueira      5
H